# What we do here

- Load items dataframe
- Embed items 
- Compute cosine similarity matrix per scale
- Construct final data frame with the following columns
    - Row =  scale name
    - Col1 = cosine similarity matrix
    - Col2 = scale id


In [24]:
#source Jonas/bin/activate 
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

In [6]:
df = pd.read_csv('items.csv')
df.head()

,item_id,scale_id,item_text,source_table
0,1,360emergencymed_azami_2024,She/he performs the clinical examinations acco...,360emergencymed_azami_2024
1,2,360emergencymed_azami_2024,The student provides accurate and authoritativ...,360emergencymed_azami_2024
2,3,360emergencymed_azami_2024,Uses appropriate communication skills while ta...,360emergencymed_azami_2024
3,4,360emergencymed_azami_2024,"She/he patiently, timely, and correctly perfor...",360emergencymed_azami_2024
4,5,360emergencymed_azami_2024,She/he knows the correct use of the medical eq...,360emergencymed_azami_2024


In [33]:
#drop empty items
df = df[~df['item_text'].isna()]

In [34]:
# First we create a list of models (all multilinguals here)
models = ['nli-distilroberta-base-v2',
          'paraphrase-multilingual-mpnet-base-v2',
          'paraphrase-multilingual-MiniLM-L12-v2',
          'intfloat/multilingual-e5-base',
          'LaBSE'] #consider adding the finetuned model for psicometrista

# Create an empty data frame, which we will then populate with the different type of embeddings
facet_embeddings_sentences = pd.DataFrame()

for mod in models:
    model = SentenceTransformer(mod) #call the model
    item_embed = [] #create list for item-level embed
    item_embed_rev = [] #create list for item-level embed accounting for sign
    for item in range(0,len(df['item_id'])): #loop over all the items
    #encode items
        item_embed.append(model.encode(df['item_text'].iloc[item]))
    df[mod + '_embeddings'] = item_embed #then, we append the two item-level embeddings list and give them a name based on the model we used

/var/folders/rw/sdqftfpx3wb1f0fk7cwkfcfw0000gn/T/ipykernel_32684/2067765195.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[mod + '_embeddings'] = item_embed #then, we append the two item-level embeddings list and give them a name based on the model we used
/var/folders/rw/sdqftfpx3wb1f0fk7cwkfcfw0000gn/T/ipykernel_32684/2067765195.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[mod + '_embeddings'] = item_embed #then, we append the two item-level embeddings list and give them a name based o

In [69]:
# Add openai embeddings
import os
from dotenv import load_dotenv
from openai import OpenAI, AuthenticationError


def _get_openai_client():
    api_key = os.environ("OPENAI_API_KEY")    
    if not api_key:
        print("OPENAI_API_KEY is not set. Add it to your .env file for local dev, "
                 "or as a Secret in Hugging Face Spaces.")
        raise RuntimeError("Missing OPENAI_API_KEY")
    return OpenAI(api_key=api_key)

_get_openai_client()

TypeError: '_Environ' object is not callable

In [63]:
key = os.environ.get('OPENAI_API_KEY')

In [66]:
_get_openai_client()

OPENAI_API_KEY is not set. Add it to your .env file for local dev, or as a Secret in Hugging Face Spaces.


RuntimeError: Missing OPENAI_API_KEY

In [46]:
# Add openai embeddings
import os
from dotenv import load_dotenv
from openai import OpenAI, AuthenticationError

# get api key from environment
api_key = os.environ.get("OPENAI_API_KEY")

# create OpenAI client
def create_client(api_key):
    try:
        client = OpenAI(api_key=api_key)
        client.models.list()
        return client
    except AuthenticationError:
        print("Incorrect API")
    return None

client = create_client(api_key)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable